---

# Sea Level
## Florian Frick


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time

---

## Loading the Data

`data_sealevel.csv` is a data set of global mean sea levels
* `sealevel`: list of global mean sea levels (millimeters).
* `sealevel_sigma`: list of the *uncertainty* in global mean sea levels (millimeters).

`data_temperature.csv` is a data set of global mean temperatures.
* `temperature`: list of global mean temperatures (degrees Celsius).


In [ ]:
year = []
sealevel = []
sealevel_sigma = []
temperature = []

dfSealevel = pd.read_csv("./data_sealevel.csv")
dfTemperature = pd.read_csv("./data_temperature.csv")

year = dfSealevel["year"].tolist()
sealevel = dfSealevel["sealevel"].tolist()
sealevel_sigma = dfSealevel["uncertainty"].tolist()
temperature = dfTemperature["temperature"].tolist()

### Data

In [ ]:
# temperature
fig, axes = plt.subplots(figsize=(16, 4), ncols=3)
axes[0].scatter(year,temperature)
axes[0].set_title("Temperature")
axes[0].set_xlabel("Year")
axes[0].set_ylabel("Temperature")

# sea level
axes[1].scatter(year,sealevel)
axes[1].set_title("Sea Level")
axes[1].set_xlabel("Year")
axes[1].set_ylabel("Sea level")

# sea level uncertainty
axes[2].scatter(year,sealevel_sigma)
axes[2].set_title("Sea level Uncertainty")
axes[2].set_xlabel("Year")
axes[2].set_ylabel("Uncertainty in sea level");

> The uncertainty in sea levels decreases over time. This is likely because technology has improved and measurements have become more and more accurate, reducing uncertainty. There are also probably more measurements done now so measurements can be easily verified with one other.

---

### Simple Sea Level - Temperature Model 

A simple model for temperature-driven changes in global mean sea level (GMSL) is from [Rahmstorf (2007)](http://science.sciencemag.org/content/315/5810/368).


The `slr` model takes two parameters, $\alpha$ and $T_{eq}$, and requires a time series of global mean temperatures: `slr(alpha, Teq, temperature)`.
* `alpha` is the sensitivity of sea-level changes to changes in global temperature. The units for $\alpha$ are millimeters of sea-level changes per year, or mm y$^{-1}$.
* `Teq` is the equilibrium global mean temperature, with units of degrees Celsius.
* `temperature` is the time series of global mean surface temperatures, assumed to be relative to the 1961-1990 mean.

The default parameter choices given in the Rahmstorf (2007) paper are $\alpha=3.4$ mm y$^{-1}$ and $T_{eq} = -0.5\ ^{\circ}$C.

Normalize model relative to 1961-1990 reference period:
- Compute the mean of the output of the slr model for the years from 1961-1990 (inclusive).
- Subtract this value from each entry in the "sealevel" list (list returned by the slr function)

In [ ]:
def slr(alpha, Teq, temperature):
    '''sea-level emulator of Rahmstorf 2007 (DOI: 10.1126/science.1135456)
    Takes global mean temperature as forcing, and parameters:
     alpha = temperature sensitivity of sea level rise, and
     Teq   = equilibrium temperature,
    and calculates a rise/fall in sea levels, based on whether the temperature
    is warmer/cooler than the equilibrium temperature Teq.'''

    n_time = len(temperature)
    deltat = 1
    sealevel = [0]*n_time
    sealevel[0] = -134
    for t in range(n_time-1):
        sealevel[t+1] = sealevel[t] + deltat*alpha*(temperature[t]-Teq)

    return sealevel

In [ ]:
# model
model = slr(3.4,-.5,temperature)

# normalization
mean = np.mean(model[81:111])
model = [model[i] - mean for i in range(len(model))]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(year,model,'orange',label="Modelled sea level")
ax.scatter(year,sealevel, label="Observed sea level")
ax.set_title("Sea Level")
ax.set_xlabel("Year")
ax.set_ylabel("Sea level")
ax.legend();

> This model is good for recent years, but does not match the data starting at around 1950.

###   Define Objective Function

**Objective function**: joint likelihood function of the observed sea level data, given the model simulation.

For a single data point in year $i$, $y_i$, with associated uncertainty $\sigma_i$, we can assume the likelihood for our model simulation in year $i$, $\eta_i$, follows a normal distribution centered at the data point.  The model simulation is a **deterministic** result of our parameter choices $\alpha$ and $T_{eq}$, so the likelihood is:

$$L(y_i \mid \alpha, T_{eq}) = \dfrac{1}{\sqrt{2 \pi} \sigma_i} e^{-\dfrac{(\eta_i(\alpha, T_{eq}) - y_i)^2}{2\sigma_i^2}}$$

The **joint log-likelihood**, is the natural logarithm of the joint likelihood function, or the product of all likelihoods associated with the individual data points. Assuming the observational data ($y_i$) are all independent, then the joint log-likelihood is:

$$l(\mathbf{y} \mid \alpha, T_{eq}) = -\dfrac{N}{2} \log{(2\pi)} - \sum_{i=1}^N \log{(\sigma_i)} - \dfrac{1}{2}\sum_{i=1}^N \left( \dfrac{\eta_i(\alpha, T_{eq}) - y_i}{\sigma_i} \right)^2$$

where, $\mathbf{y} = [y_1, y_2, \ldots, y_N]$ is the vector of sea level observations, $\eta(\alpha, T_{eq}) = [\eta_1(\alpha, T_{eq}), \eta_2(\alpha, T_{eq}), \ldots, \eta_N(\alpha, T_{eq})]$ is the vector of `slr` model output when the parameter values $\alpha$ and $T_{eq}$ are used, and $N$ is the number of observations we have.

In [ ]:

def log_likelihood(parameters, obs_temp=temperature, obs_mu=sealevel, obs_sigma=sealevel_sigma):
    ''' parameters = [\alpha, \T_eq]
        obs_temp = time series of observed global mean temperatures, used to run slr model
        obs_mu = time series of observed values, used for comparison against model
        obs_sigma = time series of uncertainties in observational data
        returns joint log-likelihood of the given model'''
    
    model = slr(alpha=parameters[0], Teq=parameters[1], temperature=temperature)
    
    # normalize
    reference = (year.index(1961), year.index(1990))
    model -= np.mean(model[reference[0]:(reference[1]+1)])

    return np.sum([np.log(stats.norm.pdf(x=model, loc=obs_mu, scale=obs_sigma))])

### Hill Climbing

In [ ]:
class State:
    def __init__(self, alpha, Teq, value):
        # self.alpha = alpha
        # self.Teq = Teq
        self.node = [alpha, Teq]
        self.value = value     

class Problem():
    def __init__(self, initial, objective_function, stepsize_alpha, stepsize_Teq):
        self.current_state = initial
        self.objective_function = objective_function
        self.stepsize_alpha = stepsize_alpha
        self.stepsize_Teq = stepsize_Teq

    def moves(self):
        '''return all possible moves to make from the current_state'''
        all_moves = []
        all_moves.append([round(sum(x),2) for x in zip(self.current_state.node, [self.stepsize_alpha,0])])
        all_moves.append([round(sum(x),2) for x in zip(self.current_state.node, [-1*self.stepsize_alpha,0])])
        all_moves.append([round(sum(x),2) for x in zip(self.current_state.node, [0,self.stepsize_Teq])])
        all_moves.append([round(sum(x),2) for x in zip(self.current_state.node, [0,-1*self.stepsize_Teq])])
        all_moves.append([round(sum(x),2) for x in zip(self.current_state.node, [self.stepsize_alpha,self.stepsize_Teq])])
        all_moves.append([round(sum(x),2) for x in zip(self.current_state.node, [-1*self.stepsize_alpha,-1*self.stepsize_Teq])])
        all_moves.append([round(sum(x),2) for x in zip(self.current_state.node, [-1*self.stepsize_alpha,self.stepsize_Teq])])
        all_moves.append([round(sum(x),2) for x in zip(self.current_state.node, [self.stepsize_alpha,-1*self.stepsize_Teq])])
        return all_moves
        
        
class Problem_hillclimb(Problem):
    def best_move(self):
        '''return the best move possible from the current_state'''
        all_moves = self.moves()
        obj_func = [self.objective_function(move) for move in all_moves]
        best = all_moves[max(zip(obj_func, range(len(obj_func))))[1]]
        return best, np.max(obj_func)
    
def hill_climb(problem, n_iter):
    for k in range(n_iter):
        nextMove, nextValue = problem.best_move()
        if nextValue <= problem.current_state.value:
            return problem.current_state
        #print(problem.current_state, nextMove)
        problem.current_state.node, problem.current_state.value = nextMove, nextValue
    print('Could not reach solution in n iterations')
    return False

In [ ]:
initial_state = State(3.4, -0.5, log_likelihood([3.4, -0.5]))
problem = Problem_hillclimb(initial_state, log_likelihood, 0.01, 0.01)
# Hill Climb
out = hill_climb(problem, n_iter=1000)

print("Calibrated [alpha, T_eq]:", out.node)
print("Log-likelihood value:", out.value)
print("Uncalibrated:", [3.4, -.5], log_likelihood([3.4,-0.5]))


calibrated_model =slr(out.node[0],out.node[1], temperature)
# normalize
mean = np.mean(calibrated_model[81:111])
calibrated_model = [calibrated_model[i] - mean for i in range(len(calibrated_model))]


fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(year,model,'orange',label="Uncalibrated model")
ax.plot(year,calibrated_model,'r',label="Calibrated model")
ax.scatter(year,sealevel, label="Observed sea level")
ax.set_title("Sea Level")
ax.set_xlabel("Year")
ax.set_ylabel("Sea level")
ax.legend();

###  Simulated annealing

In [ ]:
class Problem_annealing(Problem):
    def __init__(self, initial, objective_function, schedule_function, stepsize_alpha, stepsize_Teq):
        Problem.__init__(self, initial, objective_function, stepsize_alpha, stepsize_Teq)
        self.schedule_function = schedule_function
        
    def random_move(self):
        '''return a random move, possible from the current_state'''
        all_moves = self.moves()
        next_move = np.random.multivariate_normal(self.current_state.node, [[self.stepsize_alpha,0],[0,self.stepsize_Teq]])
        next_move = [round(next_move[0],2),round(next_move[1],2)]
        # next_move = all_moves[np.random.randint(low=0, high=len(all_moves))]
        return next_move, self.objective_function(next_move)

def simulated_annealing(problem, n_iter):
    current = problem.current_state
    
    for t in range(n_iter):
        temperature = problem.schedule_function(t)
        nextMove, nextValue = problem.random_move()
        delta_obj = nextValue - current.value
        
        if delta_obj > 0:
            problem.current_state.node, problem.current_state.value = nextMove, nextValue
        else:
            p_accept = np.exp(delta_obj/temperature)
            accept = np.random.choice([True,False],p=[p_accept,1-p_accept])
            if accept:
                problem.current_state.node, problem.current_state.value = nextMove, nextValue
    return problem.current_state

# Temperature schedule
def schedule(time):
    C = 20
    p = 0.7
    temperature = C/(time+1)**p
    return temperature

In [ ]:
initial_state = State(3.4, -0.5, log_likelihood([3.4, -0.5]))
problem = Problem_annealing(initial_state, log_likelihood, schedule, 0.01, 0.01)
# Anneal
out = simulated_annealing(problem, n_iter=1000)

print("Calibrated [alpha, T_eq]:", out.node)
print("Log-likelihood value:", out.value)
print("Uncalibrated:", [3.4, -.5], log_likelihood([3.4,-0.5]))


calibrated_model =slr(out.node[0],out.node[1], temperature)
# normalize
mean = np.mean(calibrated_model[81:111])
calibrated_model = [calibrated_model[i] - mean for i in range(len(calibrated_model))]


fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(year,model,'orange',label="Uncalibrated model")
ax.plot(year,calibrated_model,'r',label="Calibrated model")
ax.scatter(year,sealevel, label="Observed sea level")
ax.set_title("Sea Level")
ax.set_xlabel("Year")
ax.set_ylabel("Sea level")
ax.legend();

> The Rahmstorf model had values of 3.4 and -0.5 for $\alpha$ and $T_{eq}$, respectively.

> Hill climbing gave values of 2.08 and -0.9 for $\alpha$ and $T_{eq}$, respectively.

> Simulated annealing gave values of 1.92 and -0.97 for $\alpha$ and $T_{eq}$, respectively.

> Both models decreased both parameter values from the initial Rahmstorf ones. Both models fit the data better than the Rahmstorf model, are incredibly similar to one another, and neither seems to have a statistically significant advantage over the other in this situation.